In [7]:
import geopandas as gpd 
import pandas as pd
from shapely.geometry import MultiLineString
import json,re
gdb = r"E:\工作文档\gdb数据\SSPB1_1.gdb"
ZD = gpd.read_file(r"E:\exploitation\webpython\test\test.shp")
ZD.to_file('./test/test.geodatabase',driver="SQLite")

In [4]:
# 编宗地代码
for djq in df.DJZQDM.drop_duplicates():
    if djq[:9] == '500118001':
        n = 1
        for index,row in df[df.DJZQDM == djq].iterrows():
            df.loc[index,'宗地代码'] = f"{djq}JA1{'0'*(4-len(str(n)))}{n}"
            n+=1
df.to_excel(r'E:\工作文档\中山路街道宗地代码整理.xlsx',index=False)

In [1]:
import geopandas as gpd 
def zdfield(tfhgdb,gdf_,save):
    tfhshp = gpd.read_file(tfhgdb,layer='ZD_SpatialJoin8')
    def tfhs(row):
        return '、'.join(set(tfhshp[tfhshp.ZDDM == row.ZDDM].NEWMAPNO.values))
    gdf_.TFH = gdf_.apply(tfhs,axis=1)
    
    gdf_.to_file(save,encoding='gb18030')

gdf = gpd.read_file(r'E:\工作文档\三教剩下社.gdb',layer='ZD')
gdf['BDCDYH'] = ''
gdf['TFH'] = ''
gdf['飞地坐落'] = ''
gdf['ZDMJ'] = 0
gdf['ZDMJ'].astype('int32')
gdf['TDZL'] = ''
gdf['FRDB'] = ''
gdf['LXDH'] = ''
gdf['ZJHM'] = ''
gdf['SZD'] = ''
gdf['SZX'] = ''
gdf['SZB'] = ''
gdf['SZN'] = ''
gdf['FDQK'] = ''
gdf['QSLY'] = ''
zdfield(r'C:\Users\Administrator\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb',gdf,r'E:\工作文档\三教剩下社宗地.shp')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_25796\708499541.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_.to_file(save,encoding='gb18030')


In [7]:
import pandas as pd
import geopandas as gdf
df  = gdf.read_file(r'C:\Program Files\feiq\Recv Files\坡度赋值.gdb',layter='坡度赋值')
field = '唯一值'
mjfield = 'Shape_Area'
save_path = r'E:\工作文档\gdb数据\最大面积.xlsx'
eq = df[field].drop_duplicates()
sa = pd.DataFrame(columns=['BH','PDJB'])
n = 0
for value in eq:
    mjmax = max(df[df[field] == value][mjfield])
    
    sa.loc[n] = [value,df[(df[field] == value) & (df[mjfield] == mjmax)].PDJB.values[0]]
    n+=1
sa.to_excel(save_path,index=False)

In [11]:
import geopandas as gpd 
gdf = gpd.read_file(r'E:\工作文档\ZSL.gdb',layer='界址点成果')
zddms = gdf.ZDDM.drop_duplicates()
zddmlist = []
def a(row):
    jadh = gdf[gdf.ZDDM == row].JZD_NEW
    if 'J1' not in list(jadh):
        zddmlist.append(row)
zddms.map(a)
print(set(zddmlist))

{'500118001018JA10048'}


In [4]:
import fiona
fiona.supported_drivers  

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [ ]:
lay = fiona.listlayers(r'E:\exploitation\webpython\test\500106沙坪坝区作业数据_1.gdb\data')
lay

In [10]:
from fiona import collection
# 读取shp文件
with collection(r'E:\exploitation\webpython\test\500106沙坪坝区作业数据_1.gdb') as src:
    # 获取shp文件的层名
    layers = [layer['name'] for layer in src.schema['layers']]
    print(layers)

KeyError: 'layers'

In [1]:
from pk.Ownership import Ownership
ow = Ownership(r'E:\工作文档\gdb数据\三教13个村\SJSSGC.gdb')

In [2]:
for res in ow.add_jzx_all():
    print(res)

正在生成:500118017002JA10021


In [13]:
import fiona  
  
# 打开gdb数据库  
gdb_path = "E:\\exploitation\\webpython\\test\\500106沙坪坝区作业数据_1.gdb"  
driver = "OpenFileGDB"  
gdb_database = fiona.open(gdb_path, "r", driver=driver)  
  
# 获取所有数据集名称  
dataset_names = [dataset.split("=")[1] for dataset in gdb_database.meta['DATASET']]  
  
# 打印数据集名称  
for name in dataset_names:  
    print(name)  
  
# 关闭gdb数据库  
gdb_database.close()


KeyboardInterrupt: 

In [14]:
import pyodbc  
  
# 连接gdb数据库  
gdb_path = "E:\\exploitation\\webpython\\test\\500106沙坪坝区作业数据_1.gdb"  
driver = '{EsrI.GDB.ArcObjects}' # 根据你的gdb数据库驱动程序进行修改  
conn = pyodbc.connect('DRIVER={};GDB={};'.format(driver, gdb_path))  
  
# 执行SQL查询获取数据集名称  
cursor = conn.cursor()  
cursor.execute("SELECT name FROM sys.sysdataset") # 使用你的gdb数据库系统表和字段进行修改  
datasets = [row[0] for row in cursor.fetchall()]  
  
# 打印数据集名称  
for dataset in datasets:  
    print(dataset)  
  
# 关闭数据库连接  
conn.close()

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC 驱动程序管理器] 未发现数据源名称并且未指定默认驱动程序 (0) (SQLDriverConnect)')